# Import Libraries

In [1]:
import crewai
import crewai_tools
import arxiv

C:\Users\ASUS\AppData\Roaming\Python\Python310\site-packages\pydantic\fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [2]:
from dotenv import load_dotenv
import os

# 2. Configurate Env Variables

In [3]:
# Load .env file
load_dotenv()

True

In [4]:
from langchain_openai import ChatOpenAI
# Set up OpenRouter configuration
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")


In [5]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=openrouter_api_key,
)

completion = client.chat.completions.create(
  extra_headers={
    "HTTP-Referer": "<YOUR_SITE_URL>", # Optional. Site URL for rankings on openrouter.ai.
    "X-Title": "<YOUR_SITE_NAME>", # Optional. Site title for rankings on openrouter.ai.
  },
  extra_body={},
  model="openai/gpt-oss-20b:free",
  messages=[
      {"role": "system", "content": "You are to imitate Simon Sinek."},
      {"role": "user", "content": "Hello, tell me what makes good leaders?"}
  ]
)
print(completion.choices[0].message.content)

Good leaders don’t get ahead because they’re better at the details—they get ahead because they’re better at *people*. It all starts with a clear, compelling *why*.  

1. **They define a purpose profit or performance.**  
   The *why* gives the team a reason to show up every day, even when the work is hard. When people understand “why we’re doing this,” they align their personal purpose with the organization’s purpose, creating a shared commitment that is far stronger than any metric.

2. **They build safety and trust.**  
   A leader’s first responsibility is to create an environment where people can step out of their shells. Fitzpatrick’s research on “psychological safety” shows that teams that feel safe are the ones that innovate, grow, and actually deliver extraordinary results. If people fear judgment, they never speak up, and if they can’t speak up, they never learn.

3. **They practice radical empathy.**  
   Leadership is a conversation, not a monologue. Listening is an act of r

# 3. Build a Custom ArXiv Search Tool

In [6]:
# Importing crewAI tools
from crewai_tools import (
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)

# Set up API keys for Serper
serper_api_key = os.getenv("SERPER_API_KEY") # serper.dev API key

In [7]:
# Instantiate tools
docs_tool = DirectoryReadTool(directory='documents') # Tool for reading documents from a specified directory
file_tool = FileReadTool() # Tool for reading individual files
search_tool = SerperDevTool() # Tool for performing web searches using the Serper API

In [8]:
#DEFAULTS TO OPENAI EMBEDDING MODEL
# web_rag_tool = WebsiteSearchTool() # Tool for searching and extracting information from websites

#Configure Google Embedding Model and Summarization Model
os.getenv("GOOGLE_API_KEY")

web_rag_tool = WebsiteSearchTool(
    config=dict(
        llm=dict(
            provider="google", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="gemini-2.5-flash",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="google", # or openai, ollama, ...
            config=dict(
                model="models/embedding-001",
                task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

C:\Users\ASUS\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\ASUS\AppData\Roaming\Python\Python310\site-packages\alembic\config.py:592: DeprecationWarning: No path_separator found in configuration; falling back to legacy splitting on spaces, commas, and colons for prepend_sys_path.  Consider adding path_separator=os to Alembic config.
  util.warn_deprecated(


In [9]:
from typing import Type, List
from pydantic import BaseModel, Field 
from crewai.tools import BaseTool 

import arxiv
import time
import datetime

class FetchArxivPapersInput(BaseModel):
    """Input schema for FetchArxivPapersTool."""
    target_date: datetime.date = Field(..., description="Target date to fetch papers for.")

class FetchArxivPapersTool(BaseTool):
    name: str = "fetch_arxiv_papers"
    description: str = "Fetches all ArXiv papers from selected categories submitted on the target date."
    args_schema: Type[BaseModel] = FetchArxivPapersInput

    def _run(self, target_date: datetime.date) -> List[dict]:
        # List of AI-related categories. 
        # You can also include ["cs.AI", "cs.LG", "cs.CV", "cs.MA", "cs.RO"]
        AI_CATEGORIES = ["cs.CL"]

        # Define the date range for the target date
        start_date = target_date.strftime('%Y%m%d%H%M')
        end_date = (target_date + datetime.timedelta(days=1)).strftime('%Y%m%d%H%M')

        # Initialize the ArXiv client
        client = arxiv.Client(
            page_size=100,  # Fetch 100 results per page
            delay_seconds=3  # Delay between requests to respect rate limits
        )

        all_papers = []

        for category in AI_CATEGORIES:
            print(f"Fetching papers for category: {category}")

            search_query = f"cat:{category} AND submittedDate:[{start_date} TO {end_date}]"

            search = arxiv.Search(
                query=search_query,
                sort_by=arxiv.SortCriterion.SubmittedDate,
                max_results=None  # Fetch all results
            )

            # Collect results for the category
            category_papers = []
            for result in client.results(search):
                category_papers.append({
                    'title': result.title,
                    'authors': [author.name for author in result.authors],
                    'summary': result.summary,
                    'published': result.published,
                    'url': result.entry_id
                })

                # Delay between requests to respect rate limits
                time.sleep(3)

            print(f"Fetched {len(category_papers)} papers from {category}")
            all_papers.extend(category_papers)

        return all_papers

In [10]:
arxiv_search_tool = FetchArxivPapersTool()

# Step 4: Creating Agents

Agents work better when they are asked to play a role.

Based on this fact, each Agent is initialized with a:

1. Role
2. Goal
3. Backstory

In [11]:
from crewai import LLM

llm = LLM(
    model="openrouter/deepseek/deepseek-chat-v3-0324:free",
    base_url="https://openrouter.ai/api/v1",
    api_key=openrouter_api_key,
    # temperature=0.8
)

In [12]:
from crewai import Agent, Task, Crew, Process

# Agent 1: ArXiv Researcher
researcher = Agent(
    role = "Senior Researcher",
    goal = "Find the top 10 papers from the search results from ArXiv on {date}."
            "Rank them appropirately.",
    backstory = "You are a senior researcher with a deep understanding of all topics in AI and AI research."
                "You are able to identify the best research papers based on the title and abstract.",
    verbose = True,
    tools = [arxiv_search_tool],
    llm=llm
)

# The parameter verbose is set to True to see detailed logging about the agent's execution.

# The parameter tools specifies what tools an AI agent can use during its execution.

In [13]:
# Agent 2: Frontend Engineer

frontend_engineer = Agent(
    role = "Senior Frontend & AI Engineer",
    goal = "Compile the results into a HTML file.",
    backstory = "You are a competent frontend engineer writing HTML and CSS with decades of experience."
                "You have also been working with AI for decades and understand it well.",
    verbose = True,
    llm=llm
)

# The Agent class also comes with an optional parameter llm that lets you choose different LLMs for different agents.

# Step 5: Creating Tasks
Breaking down an objective into sub-objectives or Tasks and letting each AI agent work on a Task leads to better results

In [14]:
# Task for ArXiv Researcher

research_task = Task(
    description = (" Find the top 10 research papers from the search results from ArXiv on {date}."),
    expected_output = (
        "A list of top 10 research papers with the following information in the following format:"
        "- Title"
        "- Authors"
        "- Abstract"
        "- Link to the paper"
    ),
    agent = researcher,
    human_input = True,  #human_input is set to True to ensure that the agent takes our feedback on its result after working on a task.
)

In [15]:
# Task for Frontend Engineer

reporting_task = Task(
    description = ("Compile the results into a detailed report in a HTML file."),
    expected_output = (
        "An HTML file with the results in the following format:"
        "Top 10 AI Research Papers published on {date}"
        "- Title (which on clicking opens the paper in a new tab)"
        "- Authors"
        "- Short summary of the abstract (2-4 sentences)"
    ),
    agent = frontend_engineer,
    context = [research_task],      # Ensure this task relies on output of "research_task" 
    output_file = "./ai_research_report.html",
    human_input = True,
)

# Step 6: Creating the Crew

In [16]:
arxiv_research_crew = Crew(
    agents = [researcher, frontend_engineer],
    tasks = [research_task, reporting_task],
    verbose = True,
    #process=Process.sequential  # or Process.hierarchical 
)

#Process.sequential: Tasks are executed one after another in the order they're defined
#Process.hierarchical: Creates a manager-worker relationship between agents

In [18]:
# Step 7: Running the Crew and Giving Feedback on its results. 
crew_inputs = {
    "date" : "2025-08-12"
}

result = arxiv_research_crew.kickoff(inputs = crew_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 710bfc12-4500-4801-abbb-b93c5b95f175                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Researcher                                                                                       │
│                                                                                                                 │
│  Task:  Find the top 10 research papers from the search results from ArXiv on 2025-08-12.                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

C:\Users\ASUS\AppData\Roaming\Python\Python310\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Fetching papers for category: cs.CL

Fetched 64 papers from cs.CL

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Frontend & AI Engineer                                                                           │
│                                                                                                                 │
│  Task: Compile the results into a detailed report in a HTML file.                                               │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Frontend & AI Engineer                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```html                                                                                                        │
│  <!DOCTYPE html>                                                                                                │
│  <html lang="en">                                                                                               │
│  <head>                                                                                                         │
│      <meta charset="UTF-8">                                                                                     │
│      <meta name="viewport" content="width=device-width, initial-scale=1.0">                                     │
│      <title>Top 10 AI Research Papers - 2025-08-12</title>                                                      │
│      <style>                                                                                                    │
│          body {                                                                                                 │
│              font-family: Arial, sans-serif;                                                                    │
│              line-height: 1.6;                                                                                  │
│              max-width: 1000px;                                                                                 │
│              margin: 0 auto;                                                                                    │
│              padding: 20px;                                                                                     │
│              color: #333;                                                                                       │
│          }                                                                                                      │
│          h1 {                                                                                                   │
│              color: #2c3e50;                                                                                    │
│              text-align: center;                                                                                │
│              margin-bottom: 30px;                                                                               │
│          }                                                                                                      │
│          .paper {                                                                                               │
│              background-color: #f9f9f9;                                                                         │
│              border-radius: 8px;                                                                                │
│              padding: 20px;                                                                                     │
│              margin-bottom: 20px;                                                                               │
│              box-shadow: 0 2px 5px rgba(0,0,0,0.1);                                                             │
│          }                                                                                                      │
│          .title {                                                                                               │
│              font-size: 1.2em;                         

 ## Final Result: ```html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Top 10 AI Research Papers - 2025-08-12</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            line-height: 1.6;
            max-width: 1000px;
            margin: 0 auto;
            padding: 20px;
            color: #333;
        }
        h1 {
            color: #2c3e50;
            text-align: center;
            margin-bottom: 30px;
        }
        .paper {
            background-color: #f9f9f9;
            border-radius: 8px;
            padding: 20px;
            margin-bottom: 20px;
            box-shadow: 0 2px 5px rgba(0,0,0,0.1);
        }
        .title {
            font-size: 1.2em;
            font-weight: bold;
            color: #2980b9;
            margin-bottom: 10px;
        }
        .title a {
            color: inherit;
            text-decoration: no

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: ea5c25a5-f55d-4a21-b515-2b48ed0ffea9                                                                     │
│  Agent: Senior Frontend & AI Engineer                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 710bfc12-4500-4801-abbb-b93c5b95f175                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```html                                                                                          │
│  <!DOCTYPE html>                                                                                                │
│  <html lang="en">                                                                                               │
│  <head>                                                                                                         │
│      <meta charset="UTF-8">                                                                                     │
│      <meta name="viewport" content="width=device-width, initial-scale=1.0">                                     │
│      <title>Top 10 AI Research Papers - 2025-08-12</title>                                                      │
│      <style>                                                                                                    │
│          body {                                                                                                 │
│              font-family: Arial, sans-serif;                                                                    │
│              line-height: 1.6;                                                                                  │
│              max-width: 1000px;                                                                                 │
│              margin: 0 auto;                                                                                    │
│              padding: 20px;                                                                                     │
│              color: #333;                                                                                       │
│          }                                                                                                      │
│          h1 {                                                                                                   │
│              color: #2c3e50;                                                                                    │
│              text-align: center;                                                                                │
│              margin-bottom: 30px;                                                                               │
│          }                                                                                                      │
│          .paper {                                                                                               │
│              background-color: #f9f9f9;                                                                         │
│              border-radius: 8px;                                                                                │
│              padding: 20px;                                                                                     │
│              margin-bottom: 20px;                                                                               │
│              box-shadow: 0 2px 5px rgba(0,0,0,0.1);                                                             │
│          }                                                                                                      │
│          .title {                                     